# Présentation des données

:::{image} forest-cover.jpg
:::

## Introduction 

### 1. Contexte et problématique métier

Dans le domaine de la gestion environnementale, la cartographie précise des ressources forestières est un enjeu majeur. Les agences gouvernementales doivent identifier les types de forêts pour mener à bien des missions critiques, telles que la prévention des incendies ou la planification de l'exploitation du bois.

Cependant, la classification des essences d'arbres est complexe. Les frontières entre les différents types de végétation sont souvent floues et dépendent de nombreuses variables géographiques. Une erreur de classification peut avoir des conséquences lourdes, comme l'autorisation d'une coupe de bois dans une zone écologique sensible.

L'objectif de ce projet est de proposer une solution d'aide à la décision fiable. Il ne s'agit pas seulement de prédire le type de forêt, mais de quantifier l'incertitude de chaque prédiction pour sécuriser les décisions de gestion.

### 2. Présentation des données

Pour répondre à cette problématique, nous utilisons le jeu de données **Forest Cover Type** (disponible sur [Kaggle](https://www.kaggle.com/datasets/uciml/forest-cover-type-dataset)).
Ce jeu de données volumineux (plus de 500 000 échantillons) contient des mesures cartographiques telles que :
* L'altitude et la pente.
* L'exposition au soleil.
* La distance aux points d'eau et aux routes.
* La nature du sol.

La tâche consiste à réaliser une **classification multi-classe** pour prédire l'une des 7 catégories de couverture forestière (par exemple : "Spruce-Fir", "Lodgepole Pine", etc.). Ce jeu de données est particulièrement pertinent car il présente de nombreux cas ambigus où les classes se chevauchent, ce qui met à l'épreuve les algorithmes classiques.

### 3. Approche méthodologique : La prédiction conforme

Les modèles de classification traditionnels fournissent une réponse unique, parfois erronée, sans indiquer clairement leur niveau de doute.

Pour pallier ce manque, nous intégrons plusieurs méthodes issues de la **prédiction conforme**. Ces méthodes transforment la prédiction simple en un ensemble de prédictions avec une garantie statistique de fiabilité.

L'application concrète se déroule ainsi :
1.  **Zones de confiance élevée :** Si l'algorithme ne prédit qu'une seule classe (ex : {Spruce-Fir}), la cartographie est validée automatiquement.
2.  **Zones d'incertitude :** Si l'algorithme hésite, il renvoie plusieurs classes possibles (ex : {Spruce-Fir, Lodgepole Pine}). Ces zones sont alors signalées comme "à vérifier".

Cette approche permet d'optimiser le travail des experts. Au lieu de vérifier toute la carte, ils peuvent concentrer leur analyse sur les zones identifiées comme ambiguës par le modèle.

### 4. Applicabilité de la prédiction conforme

Pour bénéficier des garanties théoriques de la prédiction conforme, il faut que les données respectent l'hypothèse d'**échangeabilité**. Dans le contexte de la cartographie forestière, cette hypothèse est raisonnable car les échantillons sont collectés de manière aléatoire à travers différentes régions géographiques. De plus les données sont réparties aléatoirement entre les ensembles d'entraînement, de calibration et de test.

Toutefois, il faut rester vigilant aux effets d'**auto-corrélation spatiale** qui pourraient biaiser les résultats si des échantillons proches géographiquement sont inclus dans différents ensembles. Dans notre cas, la grande taille du jeu de données et l'absence de données géographiques précises minimisent ce risque.

## Analyse exploratoire des données

### Chargement des données

In [31]:
import polars as pl
import polars.selectors as cs
from kagglehub import KaggleDatasetAdapter, dataset_load

df = dataset_load(
    adapter=KaggleDatasetAdapter.POLARS,
    handle="uciml/forest-cover-type-dataset",
    path="covtype.csv",
).collect()
df

Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2396,153,20,85,17,108,240,237,118,837,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
2391,152,19,67,12,95,240,237,119,845,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
2386,159,17,60,7,90,236,241,130,854,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3


In [32]:
# Downcast binary columns to Boolean
binary_cols = cs.starts_with("Wilderness_Area") | cs.starts_with("Soil_Type")
df = df.with_columns(binary_cols.cast(pl.Boolean))

### Statistiques descriptives

Le jeu de données contient des variables numériques continues (ex: `Elevation`, `Aspect`, `Slope`, etc.) et des variables binaires (ex: `Wilderness_Area1`, `Soil_Type1`, etc.) mais aucune variable catégorielle autre que la variable cible `Cover_Type`.

In [33]:
df.glimpse()

Rows: 581012
Columns: 55
$ Elevation                           <i64> 2596, 2590, 2804, 2785, 2595, 2579, 2606, 2605, 2617, 2612
$ Aspect                              <i64> 51, 56, 139, 155, 45, 132, 45, 49, 45, 59
$ Slope                               <i64> 3, 2, 9, 18, 2, 6, 7, 4, 9, 10
$ Horizontal_Distance_To_Hydrology    <i64> 258, 212, 268, 242, 153, 300, 270, 234, 240, 247
$ Vertical_Distance_To_Hydrology      <i64> 0, -6, 65, 118, -1, -15, 5, 7, 56, 11
$ Horizontal_Distance_To_Roadways     <i64> 510, 390, 3180, 3090, 391, 67, 633, 573, 666, 636
$ Hillshade_9am                       <i64> 221, 220, 234, 238, 220, 230, 222, 222, 223, 228
$ Hillshade_Noon                      <i64> 232, 235, 238, 238, 234, 237, 225, 230, 221, 219
$ Hillshade_3pm                       <i64> 148, 151, 135, 122, 150, 140, 138, 144, 133, 124
$ Horizontal_Distance_To_Fire_Points  <i64> 6279, 6225, 6121, 6211, 6172, 6031, 6256, 6228, 6244, 6230
$ Wilderness_Area1                   <bool> True, True, True

In [34]:
df.describe()

statistic,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0,581012.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",2959.365301,155.656807,14.103704,269.428217,46.418855,2350.146611,212.146049,223.318716,142.528263,1980.291226,0.448865,0.051434,0.436074,0.063627,0.005217,0.012952,0.008301,0.021335,0.002749,0.011316,0.000181,0.000308,0.001974,0.056168,0.021359,0.051584,0.030001,0.001031,0.000005,0.004897,0.00589,0.003268,0.006921,0.015936,0.001442,0.057439,0.099399,0.036622,0.000816,0.004456,0.001869,0.001628,0.198356,0.051927,0.044175,0.090392,0.077716,0.002773,0.003255,0.000205,0.000513,0.026803,0.023762,0.01506,2.051471
"""std""",279.984734,111.913721,7.488242,212.549356,58.295232,1559.25487,26.769889,19.768697,38.274529,1324.19521,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.396504
"""min""",1859.0,0.0,0.0,0.0,-173.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
"""25%""",2809.0,58.0,9.0,108.0,7.0,1106.0,198.0,213.0,119.0,1024.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0
"""50%""",2996.0,127.0,13.0,218.0,30.0,1997.0,218.0,226.0,143.0,1710.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.0
"""75%""",3163.0,260.0,18.0,384.0,69.0,3328.0,231.0,237.0,168.0,2550.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.0
"""max""",3858.0,360.0,66.0,1397.0,601.0,7117.0,254.0,254.0,254.0,7173.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7.0


### Distribution des classes de la variable cible

La variable cible a une classe majoritaire (class 2) qui représente environ 50% des observations. Les classes 4, 5, 6, 7 sont sous-représentés et il peut être difficile pour un modèle de bien les prédire.

In [35]:
df.group_by("Cover_Type").len(name="Count").with_columns(
    Proportion=(pl.col("Count") / pl.sum("Count") * 100).round(1)
).sort("Count", descending=True)

Cover_Type,Count,Proportion
i64,u32,f64
2,283301,48.8
1,211840,36.5
3,35754,6.2
7,20510,3.5
6,17367,3.0
5,9493,1.6
4,2747,0.5


### Reverse OneHotEncoding

Le jeu de données est fourni avec les colonnes `Soil_Type` (40 colonnes) et `Wilderness_Area` (4 colonnes) déjà encodées en One-Hot. Les conserver sous forme de 44 colonnes booléennes séparées est inefficace pour les modèles basés sur des arbres (qui gèrent mieux les divisions catégorielles que les divisions binaires sparses).

On procède donc à un reverse One-Hot Encoding pour regrouper ces colonnes en deux colonnes catégorielles `Soil_Type` (40 catégories) et `Wilderness_Area` (4 catégories).

In [36]:
# Check mutual exclusivity of one-hot encoded columns
df.select(pl.sum_horizontal(cs.boolean())).unique().item()

2

In [ ]:
def reverse_ohe(boolean_columns: pl.Expr) -> pl.Expr:
    """Reverse one-hot encoding for a group of mutually exclusive boolean columns."""
    return (
        pl.concat_list(boolean_columns)
        .list.arg_max()
        .cast(pl.String)
        .cast(pl.Categorical)
    )


# Apply to Soil and Wilderness
df = df.with_columns(
    reverse_ohe(cs.starts_with("Soil_Type")).alias("_Soil_Type"),
    reverse_ohe(cs.starts_with("Wilderness_Area")).alias("_Wilderness_Area"),
).drop(cs.starts_with("Soil_Type") | cs.starts_with("Wilderness_Area"))
df.head()

Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type,_Soil_Type,_Wilderness_Area
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,cat,cat
2596,51,3,258,0,510,221,232,148,6279,5,"""28""","""0"""
2590,56,2,212,-6,390,220,235,151,6225,5,"""28""","""0"""
2804,139,9,268,65,3180,234,238,135,6121,2,"""11""","""0"""
2785,155,18,242,118,3090,238,238,122,6211,2,"""29""","""0"""
2595,45,2,153,-1,391,220,234,150,6172,5,"""28""","""0"""


## Sauvegarde des données nettoyées

In [ ]:
df.write_parquet("../../data/forest_cover_clean.parquet")